In [1]:
from nudging.cate import multi_dataset_cate
from nudging.simulation import generate_multi_dataset
from nudging.model import BaseBiRegressor
from sklearn.linear_model import BayesianRidge
from nudging.model.meta_regression import MetaRegressionModel
import numpy as np
from scipy.stats import spearmanr
from tqdm import tqdm

In [2]:
np.random.seed(1290834)
datasets = generate_multi_dataset(150)
base_model = BaseBiRegressor(BayesianRidge())

In [3]:
def test_multidata(base_model, datasets, train_size=10, n_retrain=10):
    corr = []
    meta_model = MetaRegressionModel(base_model)
    for _ in range(n_retrain):
        train_idx = np.random.choice(len(datasets), size=train_size, replace=False)
        test_idx = np.delete(np.arange(len(datasets)), train_idx)
        meta_model.train([datasets[x] for x in train_idx])
        for cur_id in test_idx:
            test_data = datasets[cur_id]
            cate = meta_model.predict_cate(test_data)
            corr.append(spearmanr(cate, test_data.truth["cate"]).correlation)
    return corr


In [4]:
np.mean(test_multidata(base_model, datasets, n_retrain=1))

0.34229230409688083

In [5]:
#train_sizes = np.arange(5, 75)
#all_corr = []
#for ts in tqdm(train_sizes):
#    all_corr.append(np.mean(test_multidata(model, datasets, train_size=ts)))

In [6]:
#from matplotlib import pyplot as plt
#
#plt.plot(train_sizes[:len(all_corr)], all_corr)
#plt.show()